In [1]:
# ! pipenv run jupyter nbconvert --to python get_all_txs.ipynb

In [2]:
print('get all txs')
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import google_bq_utils as bqu
sys.path.pop()

import time
import datetime

import numpy as np
import pandas as pd
import os

get all txs


In [3]:
col_list = [
        'dt','blockchain','name','layer','chain_id'
        , 'num_raw_txs', 'num_success_txs','num_qualified_txs','source'
        ]
query_name = 'dune_all_txs'
query_gas_name = 'dune_all_gas'
query_fees_name = 'dune_all_fees'
rerun_hrs = 4

In [4]:
chain_config = [
    ### #blockchain, display_name, count_func, count_block_func_param, transactions_table_param, tx_fee_func_param, tx_fee_currency_param, chain_id_param
    ['bitcoin','Bitcoin','COUNT_IF(fee > 0)','COUNT(DISTINCT block_height)','transactions','fee','BTC','NULL'],
    ['near','Near','COUNT(distinct case when gas_price > 0 then tx_hash else null end)','COUNT(DISTINCT block_height)','actions','cast(NULL as double)','NULL','397'],
    ['aptos','Aptos','COUNT_IF(gas_used>0)','COUNT(DISTINCT block_height)','user_transactions','(gas_used*gas_unit_price/1e8)','APT','NULL'],
    # ['stellar','Stellar','COUNT_IF(fee_charged>0)','COUNT(DISTINCT block_height)','fee_charged','history_transactions'], --not date partitioned
    ['kaia','Kaia','COUNT_IF(gas_price>0)','COUNT(DISTINCT block_number)','transactions','cast(NULL as double)','NULL','8217'],
    ['ton','TON','COUNT_IF(compute_gas_fees>0)','COUNT(DISTINCT block_seqno)','transactions','cast(NULL as double)','NULL','NULL'],
    ['berachain','Berachain','COUNT_IF(gas_price>0)','COUNT(DISTINCT block_number)','transactions','cast(gas_price/1e9*gas_used/1e9 as double)','BERA','80094'],
    ['sonic','Sonic','COUNT_IF(gas_price>0)','COUNT(DISTINCT block_number)','transactions','cast(gas_price/1e9*gas_used/1e9 as double)','S','146'],
]

In [5]:
trailing_days = 90
single_chain = 'none' ## if 'none' then there will be no filter applied
print('trailing days: ' + str(trailing_days))
print('single chain: ' + str(single_chain))

trailing days: 90
single chain: none


In [6]:
days_param = d.generate_query_parameter(input=trailing_days,field_name='trailing_days',dtype='number')
single_chain_param = d.generate_query_parameter(input=single_chain,field_name='single_chain',dtype='text')

In [7]:
# Run Dune - all txs
print('     dune runs')
fees_df = d.get_dune_data(query_id = 4229341, #https://dune.com/queries/4229341
    name = query_name,
    path = "outputs",
    performance="large",
    params = [days_param,single_chain_param],
    num_hours_to_rerun=rerun_hrs
)

     dune runs
Results available at https://dune.com/queries/4229341?trailing_days=90&single_chain=none
getting latest result


In [8]:
print('run dune - gas level')
gas_df = d.get_dune_data(query_id = 4758295, #https://dune.com/queries/4758295
    name = query_name,
    path = "outputs",
    performance="large",
    params = [days_param],
    num_hours_to_rerun=rerun_hrs
)

run dune - gas level
Results available at https://dune.com/queries/4758295?trailing_days=90
getting latest result


2025-04-25 09:27:26,549 INFO dune_client.api.base executing 4758295 on large cluster
2025-04-25 09:27:26,862 INFO dune_client.api.base waiting for query execution 01JSPJ3C66XHKD196BH89F7BF5 to complete: ExecutionState.PENDING (queue position: None)
2025-04-25 09:27:28,019 INFO dune_client.api.base waiting for query execution 01JSPJ3C66XHKD196BH89F7BF5 to complete: ExecutionState.PENDING (queue position: None)
2025-04-25 09:27:29,173 INFO dune_client.api.base waiting for query execution 01JSPJ3C66XHKD196BH89F7BF5 to complete: ExecutionState.PENDING (queue position: None)
2025-04-25 09:27:30,333 INFO dune_client.api.base waiting for query execution 01JSPJ3C66XHKD196BH89F7BF5 to complete: ExecutionState.PENDING (queue position: None)
2025-04-25 09:27:31,490 INFO dune_client.api.base waiting for query execution 01JSPJ3C66XHKD196BH89F7BF5 to complete: ExecutionState.PENDING (queue position: None)
2025-04-25 09:27:32,648 INFO dune_client.api.base waiting for query execution 01JSPJ3C66XHKD196

In [9]:
fees_df.sample(5)

,blockchain,chain_id,display_name,dt,median_tx_fee_native,median_tx_fee_usd,num_blocks,num_txs,tx_fee_currency,tx_fee_native,tx_fee_usd,last_updated
3234,optimism,10.0,OP Mainnet,2025-03-12,1.291854e-06,0.002435,43197,776203,0x0000000000000000000000000000000000000000,5.931566,11264.502519,2025-04-25 13:19:43.399335
2975,celo,42220.0,Celo,2025-04-23,1.560162e-03,0.001560,355,357,0xceba9300f2b948710d2653dd7b07f33a8b32118c,0.586494,0.586488,2025-04-25 13:19:43.399335
683,celo,42220.0,Celo,2025-03-14,1.428833e-03,0.000479,17279,695903,0x0000000000000000000000000000000000000000,3410.408565,1131.039800,2025-04-25 13:19:43.399335
2970,polygon,137.0,Polygon PoS,2025-01-30,3.355552e-03,0.001367,39816,3574086,0x0000000000000000000000000000000000000000,31312.782282,12804.546820,2025-04-25 13:19:43.399335
34,ink,57073.0,Ink,2025-02-11,2.069732e-07,0.000556,68350,151882,0x0000000000000000000000000000000000000000,1.023289,2730.534995,2025-04-25 13:19:43.399335


In [10]:
unique_blockchains = fees_df['blockchain'].unique().tolist()

In [11]:
print('run dune - fees metrics')
fee_metrics_df = d.get_dune_data(query_id = 4902405, #https://dune.com/queries/4902405
    name = "dune fees metrics",
    path = "outputs",
    performance="large",
    params = [days_param],
    num_hours_to_rerun=rerun_hrs
)

run dune - fees metrics
Results available at https://dune.com/queries/4902405?trailing_days=90
getting latest result


In [ ]:
print('run dune - chain level')
chain_dfs = []
for row in chain_config:
    blockchain = row[0]
    print(blockchain)
    if blockchain in unique_blockchains:
        continue
    else:
        #blockchain, display_name, count_func, gas_field, transactions_table
        blockchain_param = d.generate_query_parameter(input=blockchain,field_name='blockchain',dtype='text')
        display_name_param = d.generate_query_parameter(input=row[1],field_name='display_name',dtype='text')
        count_func_param = d.generate_query_parameter(input=row[2],field_name='count_func',dtype='text')
        count_block_func_param = d.generate_query_parameter(input=row[3],field_name='count_block_func',dtype='text')
        # gas_field_param = d.generate_query_parameter(input=row[4],field_name='gas_field',dtype='text')
        transactions_table_param = d.generate_query_parameter(input=row[4],field_name='transactions_table',dtype='text')
        tx_fee_func_param = d.generate_query_parameter(input=row[5],field_name='tx_fee_func_internal',dtype='text')
        tx_fee_currency_param = d.generate_query_parameter(input=row[6],field_name='tx_fee_currency',dtype='text')
        chain_id_param = d.generate_query_parameter(input=row[7],field_name='chain_id_param',dtype='text')

        chain_df = d.get_dune_data(query_id = 4230061, #https://dune.com/queries/4230061
            name = query_name + '_by_chain',
            path = "outputs",
            performance="large",
            params = [
                    days_param,blockchain_param,display_name_param,count_func_param,count_block_func_param
                    ,transactions_table_param,tx_fee_func_param,tx_fee_currency_param,chain_id_param
                    ],
            num_hours_to_rerun=rerun_hrs
        )
        # print(chain_df.sample(3))
        chain_dfs.append(chain_df)

chain_df_agg = pd.concat(chain_dfs)

run dune - chain level
bitcoin
Results available at https://dune.com/queries/4230061?trailing_days=90&blockchain=bitcoin&display_name=Bitcoin&count_func=COUNT_IF%28fee+%3E+0%29&count_block_func=COUNT%28DISTINCT+block_height%29&transactions_table=transactions&tx_fee_func_internal=fee&tx_fee_currency=BTC&chain_id_param=NULL
getting latest result


2025-04-25 09:28:12,805 INFO dune_client.api.base executing 4230061 on large cluster
2025-04-25 09:28:13,136 INFO dune_client.api.base waiting for query execution 01JSPJ4SC8SB9GW9DRBAFG6TJ1 to complete: ExecutionState.PENDING (queue position: None)
2025-04-25 09:28:14,295 INFO dune_client.api.base waiting for query execution 01JSPJ4SC8SB9GW9DRBAFG6TJ1 to complete: ExecutionState.PENDING (queue position: None)
2025-04-25 09:28:15,448 INFO dune_client.api.base waiting for query execution 01JSPJ4SC8SB9GW9DRBAFG6TJ1 to complete: ExecutionState.EXECUTING
2025-04-25 09:28:16,591 INFO dune_client.api.base waiting for query execution 01JSPJ4SC8SB9GW9DRBAFG6TJ1 to complete: ExecutionState.EXECUTING
2025-04-25 09:28:17,774 INFO dune_client.api.base waiting for query execution 01JSPJ4SC8SB9GW9DRBAFG6TJ1 to complete: ExecutionState.EXECUTING
2025-04-25 09:28:18,922 INFO dune_client.api.base waiting for query execution 01JSPJ4SC8SB9GW9DRBAFG6TJ1 to complete: ExecutionState.EXECUTING
2025-04-25 09:2

In [ ]:
print(fees_df.columns)
# print(chain_df_agg.columns)

In [ ]:
dune_df = pd.concat([fees_df,chain_df_agg])

# First, find chains that are only in metric_fees_df
unique_fee_metrics_df = fee_metrics_df[~fee_metrics_df['blockchain'].isin(dune_df['blockchain'])]

# Concatenate these unique chains to dune_df
dune_df = pd.concat([dune_df, unique_fee_metrics_df])

In [ ]:
if not dune_df.empty:
    dune_df['source'] = 'dune'
    dune_df['dt'] = pd.to_datetime(dune_df['dt']).dt.tz_localize(None)
    dune_df = dune_df[dune_df['dt'].dt.date < datetime.datetime.now(datetime.timezone.utc).date()]
    dune_df['chain_id'] = dune_df['chain_id'].astype(str)
    dune_df['chain_id'] = dune_df['chain_id'].astype(str).str.replace(r'\.0$', '', regex=True)

In [ ]:
if not gas_df.empty:
    gas_df['source'] = 'dune'
    gas_df['dt'] = pd.to_datetime(gas_df['dt']).dt.tz_localize(None)
    gas_df = gas_df[gas_df['dt'].dt.date < datetime.datetime.now(datetime.timezone.utc).date()]
    gas_df['chain_id'] = gas_df['chain_id'].astype(str)
    gas_df['chain_id'] = gas_df['chain_id'].astype(str).str.replace(r'\.0$', '', regex=True)

In [ ]:
# Verify that all elements are strings
assert dune_df['chain_id'].apply(type).eq(str).all(), "Not all elements are strings"
# print(dune_df['chain_id'].dtype)

In [ ]:
assert gas_df['chain_id'].apply(type).eq(str).all(), "Not all elements are strings"
assert fee_metrics_df['chain_id'].fee_metrics_df(type).eq(str).all(), "Not all elements are strings"

In [ ]:
dune_df['num_blocks'] = dune_df['num_blocks'].astype(int)
dune_df['num_txs'] = dune_df['num_txs'].astype(int)


In [ ]:
if not fee_metrics_df.empty:
    fee_metrics_df['dt'] = pd.to_datetime(fee_metrics_df['dt']).dt.tz_localize(None)
    fee_metrics_df['chain_id'] = fee_metrics_df['chain_id'].astype(str)

In [ ]:
# dune_df.sample(5)
print(dune_df.dtypes)


In [ ]:
print(f"dune_df shape: {dune_df.shape}")

In [ ]:
unique_cols = ['blockchain', 'dt','tx_fee_currency']

In [ ]:
# Identify duplicate combinations
duplicates = dune_df.duplicated(subset=unique_cols, keep=False)

# View the duplicate rows
duplicate_rows = dune_df[duplicates]

# Display the duplicate rows
print(duplicate_rows)

# Get a count of duplicates for each combination
duplicate_counts = dune_df.groupby(unique_cols).size().reset_index(name='count')
duplicate_counts = duplicate_counts[duplicate_counts['count'] > 1]

print("\nDuplicate combination counts:")
print(duplicate_counts)

In [ ]:
#BQ Upload
bqu.append_and_upsert_df_to_bq_table(dune_df, query_name, unique_keys = unique_cols)

In [ ]:
bqu.append_and_upsert_df_to_bq_table(gas_df, query_gas_name, unique_keys = ['chain_id','dt'])

In [ ]:
# fee_metrics_df[fee_metrics_df['blockchain']=='tron'].sort_values(by='dt',ascending=False)

In [ ]:
bqu.append_and_upsert_df_to_bq_table(fee_metrics_df, query_fees_name, unique_keys = ['blockchain','dt'])